In [1]:
import pandas as pd
import os
from glob import glob
from random import shuffle

In [14]:
work_dir = '/Users/amandeep/Github/table-linker/data/t2dv2'
cea_targets_path = f"{work_dir}/CEA_Round1_targets.csv" # columns are file_name, row, col
tables_path = f"{work_dir}/t2dv2-dev-input"
canonical_path = f"{work_dir}/dev-canonical"

In [15]:
!mkdir -p $canonical_path

In [16]:
def filter_tables(i_path, o_path, targets_path, t2dv2=False):
    if t2dv2:
        targets_df = pd.read_csv(targets_path, names=['file_name', 'column', 'row'], header=None)
    else:
        targets_df = pd.read_csv(targets_path, names=['file_name', 'row', 'column'], header=None)
    grouped = targets_df.groupby(by=['file_name'])
    for f, gdf in grouped:
        i_f_path = f"{i_path}/{f}.csv"
        o_f_path = f"{o_path}/{f}.csv"
        if not os.path.exists(i_f_path):
            continue
        if os.path.exists(o_f_path):
            continue
        df = pd.read_csv(i_f_path)
        columns = df.columns
        targeted_columns = ",".join([columns[i] for i in set(gdf['column'].to_list())])
        !tl canonicalize -c  "$targeted_columns" --add-context  $i_f_path > $o_f_path
    print("Done!")

In [17]:
filter_tables(tables_path, canonical_path, cea_targets_path, t2dv2=True)

canonicalize Time: 0.002440929412841797s
canonicalize Time: 0.0022428035736083984s
canonicalize Time: 0.0029730796813964844s
canonicalize Time: 0.0029981136322021484s
canonicalize Time: 0.002904176712036133s
canonicalize Time: 0.002876758575439453s
canonicalize Time: 0.0025501251220703125s
canonicalize Time: 0.002583026885986328s
canonicalize Time: 0.0027740001678466797s
canonicalize Time: 0.002226114273071289s
Done!


In [ ]:
gt_path = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/round4_gt_with_labels'
train_canonical_path  = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/train1-canonical'
dev_canonical_path  = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/dev-canonical'
all_canonical_path = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/canonical'

In [ ]:
f_list = glob(f'{gt_path}/*csv')

In [ ]:
len(f_list)

In [ ]:
f_list[:5]

In [ ]:
shuffle(f_list)

In [ ]:
f_list[:5]

In [ ]:
c = 0
for f in f_list:
    if c > 300:
        break
    f_name = f.split('/')[-1]
    c_path = f'{all_canonical_path}/{f_name}'
    if os.path.exists(c_path) and os.path.getsize(c_path) >0 :
        !cp $c_path $train_canonical_path
        c += 1
        

In [ ]:
c = 0
for f in f_list:
    if c > 50:
        break
    f_name = f.split('/')[-1]
    c_path = f'{all_canonical_path}/{f_name}'
    if os.path.exists(c_path) and os.path.getsize(c_path) > 0 and not os.path.exists(f'{train_canonical_path}/{f_name}'):
        print(c_path)
        !cp $c_path $dev_canonical_path
        c += 1